In [1]:
import timm

# LeViT 모델 불러오기 (LeViT-256 예시)
model = timm.create_model('levit_256', pretrained=True)

# 모델 구조 출력
print(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/76.0M [00:00<?, ?B/s]

LevitDistilled(
  (stem): Stem16(
    (conv1): ConvNorm(
      (linear): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (act1): Hardswish()
    (conv2): ConvNorm(
      (linear): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (act2): Hardswish()
    (conv3): ConvNorm(
      (linear): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (act3): Hardswish()
    (conv4): ConvNorm(
      (linear): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (stages): Sequential(
  

In [3]:
!pip install pytorch-ignite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.7/312.7 kB 8.6 MB/s eta 0:00:00


In [4]:
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
import ignite.metrics
import ignite.contrib.handlers

/usr/local/lib/python3.10/dist-packages/ignite/handlers/checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [5]:
DATA_DIR = './data'

IMAGE_SIZE = 32

NUM_CLASSES = 10
NUM_WORKERS = 8
BATCH_SIZE = 32
EPOCHS = 50

LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-1

validation_split = 0.1

In [6]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("device:", DEVICE)

device: cuda


In [7]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float)
])

In [8]:
train_dset = datasets.CIFAR10(root=DATA_DIR, train=True, download=True, transform=train_transform)
test_dset = datasets.CIFAR10(root=DATA_DIR, train=False, download=True, transform=transforms.ToTensor())

100%|██████████| 170M/170M [00:03<00:00, 43.8MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [9]:
train_loader = torch.utils.data.DataLoader(train_dset, batch_size=BATCH_SIZE, shuffle=True,
                                           num_workers=NUM_WORKERS, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_dset, batch_size=BATCH_SIZE, shuffle=False,
                                          num_workers=NUM_WORKERS, pin_memory=True)

In [ ]:
class swin_transformer(nn.Module):
  def __init__(self):
    super(swin_transformer, self).__init__()

In [ ]:
class ShrinkAttention(nn.Module):
  def __init__(self, embded_dim, num_heads):
    super(ShrinkAttention, self).__init__()
    self.embded_dim = embded_dim
    self.num_heads = num_heads
    self.head_dim = embded_dim // num_heads

In [ ]:
class ConvNorm(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
    super(ConvNorm, self).__init__()
    self.linear = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
    self.norm = nn.BatchNorm2d(out_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

  def forward(self, x):
    x = self.linear(x)
    x = self.norm(x)

    return x

In [ ]:
class LinearNorm(nn.Module):
  def __init__(self, in_features, out_features):
    super(LinearNorm, self).__init__()
    self.linear = nn.Linear(in_features, out_features, bias=False)
    self.norm = nn.BatchNorm1d(out_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

  def forward(self, x):
    x = self.linear(x)
    x = self.norm(x)

    return x

In [ ]:
class Downsample(nn.Module):
  def forward(self,x):
    return F.avg_pool2d(x, kernel_size=5, stride=5)
class AttentionDownsample(nn.Module):
  def __init__(self, in_features, out_features):
    super(AttentionDownsample, self).__init__()
    self.kv = LinearNorm(in_features, in_features*5)
    self.q = nn.Sequential(
        Downsample(),
        LinearNorm(in_features, in_features),
    )
    self.proj = nn.Sequential(
        nn.Hardswish(),
        LinearNorm(out_features * 3, out_features)
    )

  def forward(self, x):
    kv = self.kv(x)
    q = self.q(x)
    attention_output =
    proj = self.proj(x)

    return proj

In [ ]:
class LeViTMlp(nn.Module):
  def __init__(self ,in_features, hidden_features):
    super(LeViTMlp, self).__init__()
    self.linear1 = LinearNorm(in_features, hidden_features)
    self.linear2 = LinearNorm(hidden_features, in_features)
    self.act = nn.Hardswish()
    self.drop = nn.Dropout(p = 0.0)

  def forward(self, x):
    x = self.linear1(x)
    x = self.act(x)
    x = self.drop(x)
    x = self.linear2(x)

    return x

In [ ]:
class LeViTBlock(nn.Module):
  def __init__(self, in_features):
    super(LeViTBlock, self).__init__()
    self.drop_path1 = nn.Identity()
    self.drop_path2 = nn.Identity()
    self.mlp = LeViTMlp(in_features, in_features * 2)
    self.attn = swin_transformer(in_features)

  def forward(self, x):
    x = x + self.drop_path1(self.attn(x))
    x = x + self.drop_path2(self.mlp(x))

    return x

In [ ]:
class LeViT_swin(nn.Module):
  def __init__(self):
    super(LeViT_swin, self).__init__()
    self.stem == nn.Sequential(
        ConvNorm(3, 32, kernel_size=3, stride=2, padding=1),
        nn.Hardswish(),
        ConvNorm(32, 64, kernel_size=3, stride=2, padding=1),
        nn.Hardswish(),
        ConvNorm(64, 128, kernel_size=3, stride=2, padding=1),
        nn.Hardswish(),
        ConvNorm(128, 256, kernel_size=3, stride=2, padding=1)
    )

    self.stages = nn.Sequential(
        LeViTBlock(256),
        LeViTBlock(256),
        LeViTBlock(256),
        LeViTBlock(256),
    )